# init

In [1]:
import pandas as pd
import numpy as np
import os

# --- Percorsi dei File ---
# Path al file pickle del DataFrame
pkl_path = "/home/mmezzanzanica/project/scoring_autoint_align/data/Llama3_1-8B-Base-LXR-8x/25-llamascope-res-32k/Lajavaness/bilingual-embedding-large/oai_token-act-pair_gpt-4o-mini_embeddings.pkl"
# Path al file NPZ degli embedding di co-occorrenza
npz_path = "/home/mmezzanzanica/project/scoring_autoint_align/data/Llama3_1-8B-Base-LXR-8x/25-llamascope-res-32k/pajama_meta-llama_Llama-3.1-8B_res_Llama3_1-8B-Base-L25R-8x_checkpoints_final.safetensors_docs50k_keq256_cooccurrences.npz"

# --- Configurazione ---
# !! NUOVA OPZIONE: Specifica il numero totale di righe che il DataFrame finale dovrebbe avere.
# !! Lo script garantirà la presenza di tutti gli indici da 0 a (EXPECTED_TOTAL_ROWS - 1).
# !! Imposta a None per disattivare questa modalità e riempire solo i gap interni.
EXPECTED_TOTAL_ROWS = 32768 # Esempio: 32768. Cambia questo valore o impostalo a None.

# !! Importante: Se il tuo file NPZ ha più array, specifica la chiave corretta
# !! (nome) per l'array di embedding qui. Se None, usa il primo.
embedding_array_key = None # Esempio: 'arr_0' o 'embeddings' o None

try:
    # --- Sezione 1: Caricamento del DataFrame da PKL ---
    print(f"Caricamento DataFrame da: {pkl_path}")
    df = pd.read_pickle(pkl_path)
    print(f"DataFrame caricato con successo. Shape: {df.shape}")
    print("Colonne originali del DataFrame:", df.columns.tolist())
    print("Prime 2 righe del DataFrame originale:")
    print(df.head(2))

    # --- INIZIO CODICE MODIFICATO ---
    # --- Sezione 2: Individuazione e Inserimento Righe Mancanti ---
    print("\n--- Controllo e inserimento righe fittizie per indici mancanti ---")

    if df.empty and EXPECTED_TOTAL_ROWS is None:
        print("ATTENZIONE: Il DataFrame è vuoto e 'EXPECTED_TOTAL_ROWS' non è impostato. Impossibile procedere con il riempimento.")
    else:
        # Assicurati che la colonna 'index' sia numerica
        if not pd.api.types.is_numeric_dtype(df['index']):
            raise TypeError("La colonna 'index' non è di tipo numerico.")

        # Determina l'intervallo completo di indici in base alla nuova opzione
        if EXPECTED_TOTAL_ROWS is not None and EXPECTED_TOTAL_ROWS > 0:
            print(f"Modalità 'Numero Totale Righe' attiva. Target: {EXPECTED_TOTAL_ROWS} righe.")
            print(f"Il DataFrame verrà riempito per avere tutti gli indici da 0 a {EXPECTED_TOTAL_ROWS - 1}.")
            full_index_range = set(range(EXPECTED_TOTAL_ROWS))
        else:
            print("Modalità 'Gap Interni' attiva. Verranno riempiti solo i buchi tra l'indice minimo e massimo esistenti.")
            min_index = df['index'].min()
            max_index = df['index'].max()
            full_index_range = set(range(min_index, max_index + 1))

        # Trova gli indici mancanti
        existing_indices = set(df['index'])
        missing_indices = sorted(list(full_index_range - existing_indices))

        if not missing_indices:
            print("Nessun indice mancante trovato nell'intervallo specificato.")
        else:
            print(f"Trovati {len(missing_indices)} indici mancanti. Inizio creazione righe fittizie...")

            # Determina la dimensione degli embedding dalla prima riga valida
            first_valid_embedding = df['embedding'].dropna().iloc[0]
            first_valid_embedding_np = np.array(first_valid_embedding)
            embedding_dim = len(first_valid_embedding)
            null_embedding = np.zeros(embedding_dim, dtype=first_valid_embedding_np.dtype)
            print(f"Dimensione degli embedding rilevata: {embedding_dim}. Il vettore nullo sarà un array di zeri di questa dimensione.")

            # Crea una lista di nuove righe
            rows_to_add = []
            for missing_idx in missing_indices:
                new_row = {
                    'index': missing_idx,
                    'embedding': null_embedding
                }
                rows_to_add.append(new_row)

            # Concatena le nuove righe al DataFrame originale
            if rows_to_add:
                new_rows_df = pd.DataFrame(rows_to_add)
                df = pd.concat([df, new_rows_df], ignore_index=True)
                # Ordina il DataFrame per 'index' e resetta l'indice di pandas
                df = df.sort_values(by='index').reset_index(drop=True)
                print(f"Aggiunte {len(rows_to_add)} righe fittizie. La nuova shape del DataFrame è: {df.shape}")

    # --- FINE CODICE MODIFICATO ---


    # --- Sezione 3: Caricamento degli embedding di co-occorrenza da NPZ ---
    print(f"\nCaricamento embedding di co-occorrenza da: {npz_path}")
    with np.load(npz_path) as data:
        available_keys = data.files
        print(f"Array disponibili nel file NPZ: {available_keys}")

        if not available_keys:
            raise ValueError("Il file .npz è vuoto.")

        # Determina quale chiave dell'array usare
        if embedding_array_key is None:
            if len(available_keys) > 1:
                print(f"Attenzione: Trovati array multipli in NPZ. Uso il primo: '{available_keys[0]}'. Specificare 'embedding_array_key' se non è corretto.")
            key_to_use = available_keys[0]
        elif embedding_array_key not in available_keys:
            raise KeyError(f"La chiave specificata '{embedding_array_key}' non è stata trovata nel file NPZ. Chiavi disponibili: {available_keys}")
        else:
            key_to_use = embedding_array_key

        print(f"Utilizzo della chiave array: '{key_to_use}'")
        cooc_embeddings_matrix = data[key_to_use]
        print(f"Matrice degli embedding di co-occorrenza caricata con successo. Shape: {cooc_embeddings_matrix.shape}")

    # --- Sezione 4: Validazione delle Shape ---
    num_df_rows = len(df)
    num_embedding_rows = cooc_embeddings_matrix.shape[0]

    print(f"\nRighe del DataFrame (dopo l'inserimento): {num_df_rows}")
    print(f"Righe degli Embedding: {num_embedding_rows}")

    if num_df_rows != num_embedding_rows:
        raise ValueError(f"Mancata corrispondenza delle shape: il DataFrame ha {num_df_rows} righe, ma la matrice degli embedding ne ha {num_embedding_rows}. Impossibile unire.")
    else:
        print("Le shape corrispondono. Procedo con l'aggiunta della colonna.")

    # --- Sezione 5: Aggiunta degli embedding come nuova colonna ---
    df['cooc_embedding'] = list(cooc_embeddings_matrix)

    # --- Sezione 6: Verifica del Risultato ---
    print("\nAggiunta con successo della colonna 'cooc_embedding'.")
    print("\nInfo del DataFrame aggiornato:")
    df.info()

    print("\nHead del DataFrame aggiornato (con la nuova colonna):")
    print(df.head())

    # Opzionale: Controlla il tipo e la shape di un embedding nella colonna
    if not df.empty:
         first_embedding = df['cooc_embedding'].iloc[0]
         print(f"\nTipo della prima voce nella colonna 'cooc_embedding': {type(first_embedding)}")
         if hasattr(first_embedding, 'shape'):
             print(f"Shape della prima voce: {first_embedding.shape}")


except FileNotFoundError as e:
    print(f"\nErrore: File non trovato.")
    print(e)
except KeyError as e:
    print(f"\nErrore: Errore di chiave durante il caricamento NPZ.")
    print(e)
except ValueError as e:
    print(f"\nErrore: Errore di valore (es. shape non corrispondenti o file vuoto).")
    print(e)
except Exception as e:
    print(f"\nSi è verificato un errore inaspettato: {e}")

# Ora la variabile 'df' contiene il tuo DataFrame aggiornato
# Puoi salvarlo se necessario:
# output_path = "percorso/al/tuo/dataframe_aggiornato.pkl"
# df.to_pickle(output_path)
# print(f"\nDataFrame aggiornato salvato in: {output_path}")

Caricamento DataFrame da: /home/mmezzanzanica/project/scoring_autoint_align/data/Llama3_1-8B-Base-LXR-8x/25-llamascope-res-32k/Lajavaness/bilingual-embedding-large/oai_token-act-pair_gpt-4o-mini_embeddings.pkl
DataFrame caricato con successo. Shape: (32657, 14)
Colonne originali del DataFrame: ['id', 'modelId', 'layer', 'index', 'authorId', 'description', 'embedding', 'typeName', 'explanationModelName', 'umap_x', 'umap_y', 'umap_cluster', 'umap_log_feature_sparsity', 'true_description']
Prime 2 righe del DataFrame originale:
                          id      modelId                  layer  index  \
0  ictut37hpv7y520yomqmb2n3z  llama3.1-8b  25-llamascope-res-32k      0   
1  k6l2dqw71ssd7if32n5c1v3dp  llama3.1-8b  25-llamascope-res-32k      1   

                    authorId  \
0  clkht01d40000jv08hvalcvly   
1  clkht01d40000jv08hvalcvly   

                                         description  \
0  [ phrases related to political competition and...   
1          [terms related to crime

In [2]:
# %% [markdown]
# ## Compute Pairwise Cosine Similarities for Embeddings
#
# This cell calculates the pairwise cosine similarity matrix for two sets of vectors stored in the DataFrame:
# 1.  The original `'embedding'` column.
# 2.  The `'cooc_embedding'` column.
#
# It uses the `cosine_similarity_matrix_gpu` function for efficient computation on the GPU if available.

# %%
# --- Imports ---
import torch
import pandas as pd
import numpy as np
from tqdm.auto import tqdm # Use auto version for better notebook integration
import gc # Garbage collector for explicit memory management

# --- Assume df is already loaded and prepared ---
# Example loading (replace with your actual df variable if it's already in memory):
# pkl_path = "pipeline/data/gemma-scope-9b-pt-res-canonical/layer_8.width_16k.canonical/new_maxcorr_original.pkl"
# npz_path = "pipeline/data/cooc-histograms/pile_google_gemma-2-9b_res_layer_8_width_16k_average_l0_99_docs50k_keq256_cooccurrences.npz"
#
# df = pd.read_pickle(pkl_path)
# with np.load(npz_path) as data:
#    cooc_embeddings_matrix = data[data.files[0]] # Use the first array
#
# if len(df) == cooc_embeddings_matrix.shape[0]:
#    df['cooc_embedding'] = list(cooc_embeddings_matrix)
#    df = df[['id', 'description', 'embedding', 'cooc_embedding']]
#    print("DataFrame prepared.")
# else:
#    raise ValueError("Shape mismatch between DataFrame and NPZ file.")

# --- GPU Setup ---
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- Helper Function (Copied from your provided code) ---
def cosine_similarity_matrix_gpu(matrix, chunk_size=1000, subset_indices=None):
    """
    Compute cosine similarities for a matrix using GPU.

    If subset_indices is provided, compute the similarity of each row in `matrix`
    with only those rows specified by subset_indices.

    Args:
        matrix: Input tensor with shape (n_samples, n_features)
        chunk_size: Size of chunks to process at once.
        subset_indices: Optional list or tensor of indices to restrict the computation to.

    Returns:
        torch.Tensor: If subset_indices is provided, returns a similarity matrix
                      with shape (n_samples, len(subset_indices)).
                      Otherwise, returns (n_samples, n_samples).
    """
    if subset_indices is not None:
        # Get the subset of rows (features) to compare against.
        subset_matrix = matrix[subset_indices]

        # Normalize both the full matrix and the subset matrix
        full_norm = torch.norm(matrix, p=2, dim=1, keepdim=True)
        normalized_matrix = matrix / (full_norm + 1e-8)
        subset_norm = torch.norm(subset_matrix, p=2, dim=1, keepdim=True)
        normalized_subset_matrix = subset_matrix / (subset_norm + 1e-8)

        n = matrix.shape[0]
        k = len(subset_indices)
        similarity_matrix = torch.zeros((n, k), dtype=torch.float32, device=device)

        # Use tqdm.auto for notebook-friendly progress bar
        for i in tqdm(range(0, n, chunk_size), desc="Computing cosine similarities for subset"):
            end = min(i + chunk_size, n)
            chunk = normalized_matrix[i:end]
            similarity_matrix[i:end] = torch.mm(chunk, normalized_subset_matrix.t())
        return similarity_matrix
    else:
        # Original full matrix computation
        n = matrix.shape[0]
        similarity_matrix = torch.zeros((n, n), dtype=torch.float32, device=device)
        norm = torch.norm(matrix, p=2, dim=1, keepdim=True)
        normalized_matrix = matrix / (norm + 1e-8)

        # Use tqdm.auto for notebook-friendly progress bar
        for i in tqdm(range(0, n, chunk_size), desc="Computing pairwise cosine similarities"):
            end = min(i + chunk_size, n)
            chunk = normalized_matrix[i:end]
            similarity_matrix[i:end] = torch.mm(chunk, normalized_matrix.t())
        return similarity_matrix
# --- End of Helper Function ---

# --- Main Computation ---
sim_matrix_embedding = None
sim_matrix_cooc = None

try:
    # --- Ensure DataFrame is loaded ---
    if 'df' not in locals() or not isinstance(df, pd.DataFrame):
         raise NameError("DataFrame 'df' not found or is not a Pandas DataFrame. Please load it first.")
    if not all(col in df.columns for col in ['embedding', 'cooc_embedding']):
        raise KeyError("DataFrame must contain 'embedding' and 'cooc_embedding' columns.")
    if df.empty:
        raise ValueError("DataFrame is empty.")

    # --- Clear GPU cache ---
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect() # Run garbage collection

    # --- Process 'embedding' column ---
    print("\nProcessing 'embedding' column...")
    embeddings_list = df['embedding'].tolist()
    if not embeddings_list:
         raise ValueError("DataFrame 'embedding' column is empty.")

    # Convert list of numpy arrays/lists to a stacked PyTorch tensor
    try:
        embedding_tensor = torch.stack([torch.tensor(e, dtype=torch.float32) for e in embeddings_list]).to(device)
        print(f"Created 'embedding' tensor with shape: {embedding_tensor.shape} on device: {embedding_tensor.device}")
    except Exception as e:
        print(f"Error converting 'embedding' column to tensor. Check if all embeddings have the same length.")
        raise e

    # Compute cosine similarity
    sim_matrix_embedding = cosine_similarity_matrix_gpu(embedding_tensor)
    print(f"Computed similarity matrix for 'embedding'. Shape: {sim_matrix_embedding.shape}")

    # --- Clean up memory ---
    del embedding_tensor # Remove the large tensor variable
    if torch.cuda.is_available():
        torch.cuda.empty_cache() # Clear GPU cache again
        gc.collect()

    # # --- Process 'cooc_embedding' column ---
    # print("\nProcessing 'cooc_embedding' column...")
    # cooc_embeddings_list = df['cooc_embedding'].tolist()
    # if not cooc_embeddings_list:
    #      raise ValueError("DataFrame 'cooc_embedding' column is empty.")

    # # Convert list of numpy arrays/lists to a stacked PyTorch tensor
    # try:
    #     cooc_embedding_tensor = torch.stack([torch.tensor(c, dtype=torch.float32) for c in cooc_embeddings_list]).to(device)
    #     print(f"Created 'cooc_embedding' tensor with shape: {cooc_embedding_tensor.shape} on device: {cooc_embedding_tensor.device}")
    # except Exception as e:
    #     print(f"Error converting 'cooc_embedding' column to tensor. Check if all embeddings have the same length.")
    #     raise e

    # # Compute cosine similarity
    # sim_matrix_cooc = cosine_similarity_matrix_gpu(cooc_embedding_tensor)
    # print(f"Computed similarity matrix for 'cooc_embedding'. Shape: {sim_matrix_cooc.shape}")

    # # --- Clean up memory ---
    # del cooc_embedding_tensor
    # if torch.cuda.is_available():
    #     torch.cuda.empty_cache()
    #     gc.collect()

    # print("\nSuccessfully computed both similarity matrices.")
    # # The results are stored in sim_matrix_embedding and sim_matrix_cooc (as PyTorch tensors on the specified device)

except NameError as e:
    print(f"Error: {e}")
except KeyError as e:
    print(f"Error: {e}")
except ValueError as e:
     print(f"Error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    import traceback
    traceback.print_exc()

# %% [markdown]
# You can now use the resulting tensors `sim_matrix_embedding` and `sim_matrix_cooc` for further analysis. Remember they are currently residing on the GPU if CUDA is enabled. You might want to move them to the CPU using `.cpu()` for easier interaction with libraries like NumPy or Pandas.
#
# Example:
# ```python
# # Move to CPU and convert to NumPy (optional)
# sim_matrix_embedding_np = sim_matrix_embedding.cpu().numpy()
# sim_matrix_cooc_np = sim_matrix_cooc.cpu().numpy()
# print(sim_matrix_embedding_np.shape, sim_matrix_cooc_np.shape)
# ```

# %%
# Example: Display shapes of the resulting tensors
if sim_matrix_embedding is not None:
    print(f"\nFinal shape of sim_matrix_embedding: {sim_matrix_embedding.shape}")
    print(f"Device of sim_matrix_embedding: {sim_matrix_embedding.device}")
if sim_matrix_cooc is not None:
    print(f"Final shape of sim_matrix_cooc: {sim_matrix_cooc.shape}")
    print(f"Device of sim_matrix_cooc: {sim_matrix_cooc.device}")

/home/mmezzanzanica/project/scoring_autoint_align/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu

Processing 'embedding' column...
Created 'embedding' tensor with shape: torch.Size([32768, 1024]) on device: cpu


Computing pairwise cosine similarities: 100%|██████████| 33/33 [00:02<00:00, 13.85it/s]


Computed similarity matrix for 'embedding'. Shape: torch.Size([32768, 32768])

Final shape of sim_matrix_embedding: torch.Size([32768, 32768])
Device of sim_matrix_embedding: cpu


In [3]:
df

,id,modelId,layer,index,authorId,description,embedding,typeName,explanationModelName,umap_x,umap_y,umap_cluster,umap_log_feature_sparsity,true_description,cooc_embedding
0,ictut37hpv7y520yomqmb2n3z,llama3.1-8b,25-llamascope-res-32k,0,clkht01d40000jv08hvalcvly,[ phrases related to political competition and...,"[0.01134653203189373, -0.02747834473848343, -0...",[oai_token-act-pair],[gpt-4o-mini],0.0,0.0,0.0,0.0,phrases related to political competition and ...,"[592, 40, 1, 11, 0, 164, 4, 0, 18, 93, 9, 117,..."
1,k6l2dqw71ssd7if32n5c1v3dp,llama3.1-8b,25-llamascope-res-32k,1,clkht01d40000jv08hvalcvly,[terms related to crime and legal issues],"[0.03746338561177254, 0.01680091954767704, 0.0...",[oai_token-act-pair],[gpt-4o-mini],0.0,0.0,0.0,0.0,terms related to crime and legal issues,"[40, 2477, 9, 42, 0, 808, 73, 0, 77, 381, 24, ..."
2,gu0kewxxlneim9n2c2wy7dzz7,llama3.1-8b,25-llamascope-res-32k,2,clkht01d40000jv08hvalcvly,[ instructions related to cooking techniques a...,"[0.04790233448147774, 0.03476620092988014, 0.0...",[oai_token-act-pair],[gpt-4o-mini],0.0,0.0,0.0,0.0,instructions related to cooking techniques an...,"[1, 9, 501, 9, 0, 68, 49, 0, 5, 67, 2, 16, 65,..."
3,f2ktlifxms3bvquwz6fms8ia5,llama3.1-8b,25-llamascope-res-32k,3,clkht01d40000jv08hvalcvly,[action verbs related to registration and site...,"[0.06106364354491234, 0.024103811010718346, 0....",[oai_token-act-pair],[gpt-4o-mini],0.0,0.0,0.0,0.0,action verbs related to registration and site ...,"[11, 42, 9, 1263, 0, 243, 61, 2, 13, 216, 7, 1..."
4,q82m0h33vrrmr59e6stu75avh,llama3.1-8b,25-llamascope-res-32k,4,clkht01d40000jv08hvalcvly,[ phrases related to substance use and its eff...,"[0.0539846196770668, 0.01433128584176302, -0.0...",[oai_token-act-pair],[gpt-4o-mini],0.0,0.0,0.0,0.0,phrases related to substance use and its effects,"[0, 0, 0, 0, 41, 13, 0, 0, 2, 4, 2, 4, 9, 5, 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32763,d91rgyuqrmxfvyja9k8g1svaz,llama3.1-8b,25-llamascope-res-32k,32763,clkht01d40000jv08hvalcvly,[references to significant violent events and ...,"[0.07495604455471039, 0.039005376398563385, -0...",[oai_token-act-pair],[gpt-4o-mini],0.0,0.0,0.0,0.0,references to significant violent events and t...,"[102, 1013, 13, 113, 3, 2068, 266, 0, 127, 858..."
32764,fzjpn2e2pco94ezwi5xl9qjgd,llama3.1-8b,25-llamascope-res-32k,32764,clkht01d40000jv08hvalcvly,[ references to product design and development...,"[0.003819491248577833, 0.0064230007119476795, ...",[oai_token-act-pair],[gpt-4o-mini],0.0,0.0,0.0,0.0,references to product design and development ...,"[3, 42, 25, 32, 1, 732, 175, 0, 42, 420, 8, 31..."
32765,mqo1xj6rjky0pqpffnusfvcpr,llama3.1-8b,25-llamascope-res-32k,32765,clkht01d40000jv08hvalcvly,[ keywords related to event handling and manip...,"[0.03350266069173813, 0.010843259282410145, -0...",[oai_token-act-pair],[gpt-4o-mini],0.0,0.0,0.0,0.0,keywords related to event handling and manipu...,"[0, 0, 0, 3, 0, 4, 0, 8, 1, 50, 0, 9, 44, 1, 0..."
32766,yfrgt1mmithan3lqsahy0fv8w,llama3.1-8b,25-llamascope-res-32k,32766,clkht01d40000jv08hvalcvly,[ references to competitive matches and their ...,"[0.048039622604846954, 0.027409354224801064, -...",[oai_token-act-pair],[gpt-4o-mini],0.0,0.0,0.0,0.0,references to competitive matches and their o...,"[2, 7, 0, 5, 0, 8, 6, 0, 0, 68, 2, 5, 10, 12, ..."


In [4]:
#load scores json
import json
scores = json.load(open('/Users/antonioserino/Documents/scoring_autoint_align/pipeline/data/eleuther_fuzz_gemini-2.0-flash_scores.json'))

values_list = list(scores.values())
df["scores"] = pd.Series(values_list)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/antonioserino/Documents/scoring_autoint_align/pipeline/data/eleuther_fuzz_gemini-2.0-flash_scores.json'

# Simil

In [ ]:
import math
from IPython.display import display, HTML

ref_feature_index = 
top_n_features = 50

# --- Function to Calculate Phi Coefficient ---
def calculate_phi_coefficient(n_ii, n_jj, n_ij, N):
    """Calculates the Phi coefficient (Φ) from counts."""
    n_ii, n_jj, n_ij, N = float(n_ii), float(n_jj), float(n_ij), float(N)
    if N == 0: return 0.0 # Prevent division by zero if N is 0
    n11 = n_ij
    n10 = n_ii - n_ij
    n01 = n_jj - n_ij
    # Ensure n00 is not negative if counts are inconsistent
    n00 = max(0.0, N - n_ii - n_jj + n_ij)
    ni_dot = n_ii
    n_dot_j = n_jj
    n0_dot = N - ni_dot
    n_dot_0 = N - n_dot_j
    try:
        denominator = math.sqrt(ni_dot * n_dot_j * n0_dot * n_dot_0)
    except ValueError:
        print(ni_dot, n_dot_j, n0_dot, n_dot_0)
    if denominator == 0:
        # If denominator is 0, implies at least one marginal is 0 or N.
        # Phi is technically undefined, but often treated as 0 association.
        # Or, if n11=N and n00=0, and others are 0, it's perfect association (phi=1)
        # For simplicity, return 0 unless it's clearly perfect positive/negative.
        # A more robust handling might be needed depending on data specifics.
        if n11 == ni_dot and n11 == n_dot_j and n00 == 0: # Perfect Positive case
             return 1.0
        if n10 == ni_dot and n01 == n_dot_j and n11 == 0: # Perfect Negative case
            return -1.0
        return 0.0

    numerator = (n11 * n00) - (n10 * n01)
    # Handle potential floating point inaccuracies leading to tiny values outside [-1, 1]
    phi = numerator / denominator
    return np.clip(phi, -1.0, 1.0)


# --- Function to Get Most Similar Features based on Phi ---
def get_most_phi_correlated_features(ref_idx, df_input, cooc_mat, N_total, top_n=10):
    """Finds features most correlated with a reference feature using Phi coefficient."""

    if 'description' not in df_input.columns and 'interpretation' not in df_input.columns:
        raise ValueError("DataFrame needs an 'interpretation' or 'description' column.")
    interpretation_col = 'description' if 'description' in df_input.columns else 'interpretation'

    if ref_idx < 0 or ref_idx >= len(df_input):
        raise IndexError(f"Reference index {ref_idx} is out of bounds.")
    if cooc_mat.shape[0] != len(df_input) or cooc_mat.shape[1] != len(df_input):
        raise ValueError("Shape of cooc_mat must match DataFrame length.")

    # Extract occurrence counts once
    occurrence_counts = np.diag(cooc_mat).astype(np.float32)
    cooc_mat_float = cooc_mat.astype(np.float32) # Ensure float for calculations

    ref_n_ii = occurrence_counts[ref_idx]

    if ref_n_ii == 0:
        print(f"Warning: Reference feature {ref_idx} never occurred. Phi is likely 0 for all pairs.")
        # Create a copy to avoid modifying original df
        df_result = df_input.copy()
        df_result['phi_similarity'] = 0.0 # Assign 0 similarity
        # Still sort by index or another column if needed, but similarity is uniform
        return df_result.iloc[[ref_idx]].append(df_result.drop(ref_idx).head(top_n -1))


    phi_scores = []
    indices = []
    interpretations = []

    # Iterate through all features to calculate Phi with the reference
    for j in range(len(df_input)):
        if j == ref_idx: # Skip self-comparison calculation but store placeholder
            phi_score = 1.0 # Or np.nan if you prefer not to include self in top N
        else:
            n_jj = occurrence_counts[j]
            n_ij = cooc_mat_float[ref_idx, j]
            phi_score = calculate_phi_coefficient(ref_n_ii, n_jj, n_ij, N_total)

        phi_scores.append(phi_score)
        indices.append(df_input.index[j]) # Store original index
        interpretations.append(df_input.iloc[j][interpretation_col])


    # Create a temporary DataFrame for sorting
    df_scores = pd.DataFrame({
        'original_index': indices,
        'phi_similarity': phi_scores,
        interpretation_col: interpretations
    })

    # Sort by Phi score in descending order (higher positive Phi = more similar)
    df_sorted = df_scores.sort_values(by='phi_similarity', ascending=False)

    # Return the top N rows (including self if phi_score was set to 1.0 and it remains top)
    return df_sorted.head(top_n)

# --- Example Usage ---
interpretation_col = 'description' if 'description' in df.columns else 'interpretation'
N_total_chunks = 71687

print(f"\nFinding top {top_n_features} features functionally similar (by Phi) to feature at index {ref_feature_index}:")
print(f"Reference Interpretation: {df.loc[ref_feature_index][interpretation_col]}")

df_similar_phi = get_most_phi_correlated_features(
    ref_feature_index,
    df,
    cooc_embeddings_matrix,
    N_total_chunks,
    top_n=top_n_features
)

# Display results nicely
print("\nMost Functionally Similar Features (by Phi Coefficient):")
print("-" * 60)
# Use display(HTML(...)) for better table formatting in Jupyter/Colab
html_table = df_similar_phi.to_html(index=False, escape=False, columns=['original_index', 'phi_similarity', interpretation_col])
display(HTML(html_table))
# Or use print for plain text environments:
# print(df_similar_phi[['original_index', 'phi_similarity', interpretation_col]].to_string(index=False))
print("-" * 60)


Finding top 50 features functionally similar (by Phi) to feature at index 29280:
Reference Interpretation: [' terms related to correspondence and communication']

Most Functionally Similar Features (by Phi Coefficient):
------------------------------------------------------------


original_index,phi_similarity,description
29280,1.000000,[ terms related to correspondence and communication]
7386,0.209426,[terms related to equity and equitable practices]
22856,0.138448,[ references to comparisons or similarities between concepts or things]
24237,0.120691,[references to environmental statistics and impacts related to carbon emissions and deforestation]
18029,0.106811,[ punctuation and symbols in mathematical expressions]
18491,0.101959,[statements of mathematical conditions and their properties]
5776,0.100009,[ mathematical terms and relationships within equations and theorems]
17812,0.098584,[ terms related to equity and equality in various contexts]
16693,0.098148,[ mathematical concepts related to functions and their properties]
27111,0.097739,[mathematical and scientific terms related to functions and dynamics]


------------------------------------------------------------


In [5]:
display(HTML(df_similar_phi[['description']][0:20].to_html(index=False, escape=False)))

description
"[ instances of the word ""mock"" or its variations in the text]"
[method calls in programming that set or return values]
"[references to the movie ""Mockingjay"" and terms related to sockets]"
"[ the term ""mock"" and its variations in various contexts]"
"[ occurrences of the word ""test"" in various contexts]"
[ elements related to testing and assertion in code]
"[Occurrences of the word ""test"" in various contexts]"
[ occurrences of the underscore (_) character in the text]
[assertion statements in code]
[ terms associated with generator functions and their implementations]


# Score Calc

In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm
import math # Only needed if you keep the phi function below
from IPython.display import display, HTML

# --- Input Parameters ---
ref_feature_index = 29280  # The DataFrame index of the feature you want to inspect
top_n_features = 50    # How many neighbors you want to see
embedding_col_name = 'embedding' # Name of the column containing semantic embeddings

# --- Data Loading Placeholder ---
# IMPORTANT: Ensure df is loaded.
# df MUST have the 'embedding' column and a description/interpretation column.

# --- Example Placeholder Data (if needed) ---
if 'df' not in locals():
    N_FEATURES = 500
    EMBED_DIM = 126
    df = pd.DataFrame({
        'feature_id': range(N_FEATURES),
        'description': [f'Interpretation for feature {i}' for i in range(N_FEATURES)],
        'embedding': [np.random.rand(EMBED_DIM).astype(np.float32) for _ in range(N_FEATURES)]
    })
    print("Created dummy DataFrame for demonstration.")
# --- End Placeholder Data ---


# --- Function to Get Most Similar Features based on Cosine Similarity ---
def get_most_semantically_similar_features(ref_idx, df_input, embedding_column, top_n=10):
    """Finds features most semantically similar to a reference feature using cosine similarity."""

    if embedding_column not in df_input.columns:
         raise ValueError(f"DataFrame must contain the embedding column: '{embedding_column}'.")
    if 'description' not in df_input.columns and 'interpretation' not in df_input.columns:
        print("Warning: No 'description' or 'interpretation' column found.")
        interpretation_col = None
    else:
        interpretation_col = 'description' if 'description' in df_input.columns else 'interpretation'


    if ref_idx < 0 or ref_idx >= len(df_input):
        raise IndexError(f"Reference index {ref_idx} is out of bounds.")

    # Prepare embeddings
    embeddings_np = np.stack(df_input[embedding_column].values).astype(np.float32)
    n_features = embeddings_np.shape[0]
    ref_embedding = embeddings_np[ref_idx].reshape(1, -1)

    # Calculate cosine similarities between reference and all others
    similarities = cosine_similarity(ref_embedding, embeddings_np)[0] # Get the 1D array

    # Set self-similarity to a very low value to exclude it from top N ranking
    similarities[ref_idx] = -np.inf

    # Get indices sorted by similarity (descending)
    sorted_indices = np.argsort(-similarities)
    top_n_indices = sorted_indices[:top_n]

    # --- Prepare results DataFrame ---
    results_data = {
        'original_index': df_input.index[top_n_indices], # Get original indices
        'semantic_similarity': similarities[top_n_indices]
    }
    if interpretation_col:
        results_data[interpretation_col] = df_input.iloc[top_n_indices][interpretation_col].values

    df_similar = pd.DataFrame(results_data)

    return df_similar

# --- Example Usage ---
interpretation_col = 'description' if 'description' in df.columns else 'interpretation'
if interpretation_col not in df.columns:
    print("Warning: No interpretation/description column found for display.")
    interpretation_col = None # Prevent errors later

print(f"\nFinding top {top_n_features} features semantically similar (by Cosine Sim) to feature at index {ref_feature_index}:")
if interpretation_col:
    print(f"Reference Interpretation: {df.loc[ref_feature_index][interpretation_col]}")

# Get the similar features based on embedding similarity
df_similar_emb = get_most_semantically_similar_features(
    ref_feature_index,
    df,
    embedding_col_name, # Pass the column name
    top_n=top_n_features
)

# Display results nicely
print("\nMost Semantically Similar Features (by Embedding Cosine Similarity):")
print("-" * 60)
display_cols = ['original_index', 'semantic_similarity']
if interpretation_col:
    display_cols.append(interpretation_col)

html_table = df_similar_emb.to_html(index=False, escape=False, columns=display_cols)
display(HTML(html_table))
print("-" * 60)


Finding top 50 features semantically similar (by Cosine Sim) to feature at index 29280:
Reference Interpretation: [' terms related to correspondence and communication']

Most Semantically Similar Features (by Embedding Cosine Similarity):
------------------------------------------------------------


original_index,semantic_similarity,description
29236,0.926679,[ references to sending and receiving information or correspondence]
8330,0.877584,[communication-related terminology]
18430,0.870402,[phrases related to contact information and communication]
11677,0.867387,[ phrases related to contact and communication methods]
5257,0.845585,[ phrases related to communication and contact details]
28518,0.837193,[ references to methods of communication or submission]
20181,0.828244,[ words related to email communication]
28110,0.825932,[ phrases related to formal communication and official correspondence]
167,0.821123,[terms related to communication and discourse]
22291,0.820982,[ phrases and terms related to communication as a concept or practice]


------------------------------------------------------------


In [7]:
display(HTML(df_similar_emb[['description']][0:20].to_html(index=False, escape=False)))

description
"[ the term ""mock"" and its variations in various contexts]"
"[ instances of the word ""insult"" and its variations]"
"[ occurrences of the word ""moot"" or variations of it]"
[ instances of humor in the text]
[ instances of laughter or humorous expressions in the text]
"[instances of the word ""lol"" or its variations in the text]"
"[ instances of the word ""laugh"" or variations of it]"
[ adjectives related to humor and similarity]
[references to humor or jokes in various contexts]
"[instances of the word ""jokes.""]"


In [7]:
# rename the columns
df_similar_phi.rename(columns={'original_index': 'id', 'phi_similarity': 'phi'}, inplace=True)
df_similar_emb.rename(columns={'original_index': 'id', 'semantic_similarity': 'cosine'}, inplace=True)

In [ ]:
# rename the columns
df_similar_phi.rename(columns={'original_index': 'id', 'phi_similarity': 'phi'}, inplace=True)
df_similar_emb.rename(columns={'original_index': 'id', 'semantic_similarity': 'cosine'}, inplace=True)
#common rows between df_similar_sign_emb and df_similar_sign_funct
df_common = pd.merge(df_similar_emb, df_similar_phi, on='id', suffixes=('_emb', '_cooc'))

len(df_common)

1

In [10]:
import pandas as pd
import numpy as np
import torch  # Make sure PyTorch is installed (pip install torch)
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm
import math
import time
from IPython.display import display, HTML

# --- Configuration ---
TOP_N = 1023  # Number of top neighbors to consider for the rankings
EMBEDDING_COL = 'embedding'  # Column name for semantic embeddings
PHI_SIM_COL = 'phi_similarity_vector'  # Placeholder, not stored in df
OUTPUT_COL = 'ndcg_emb_phi'  # <— NEW: column for the NDCG scores (0–1)
ID_COL = df.index.name if df.index.name else 'index'  # Get index name or use 'index'

# --- GPU Device Setup ---
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using CUDA device: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("CUDA not available, using CPU (will be slower for Phi).")

# --- Data Loading Placeholder ---
# (unchanged ‑ ensure df, cooc_embeddings_matrix, N_total_chunks exist)
N_total_chunks = 70248  # Make sure this is correct

# --- Function to Calculate Phi Coefficient Matrix on CUDA (unchanged) ---
@torch.no_grad()
def calculate_phi_matrix_cuda(cooc_mat_np, N_total, device):
    """Calculates the full NxN Phi coefficient matrix using PyTorch on CUDA."""
    print("  Calculating Phi matrix on device:", device)
    n_features = cooc_mat_np.shape[0]
    cooc_mat_t = torch.from_numpy(cooc_mat_np.astype(np.float32)).to(device)
    N = torch.tensor(N_total, dtype=torch.float32, device=device)
    n_i_dot = torch.diag(cooc_mat_t)
    n_dot_j = n_i_dot.clone()
    n_ii_mat = n_i_dot.unsqueeze(1).expand(-1, n_features)
    n_jj_mat = n_dot_j.unsqueeze(0).expand(n_features, -1)
    n11 = cooc_mat_t
    n10 = n_ii_mat - n11
    n01 = n_jj_mat - n11
    n00 = torch.clamp(N - n_ii_mat - n_jj_mat + n11, min=0.0)
    n0_dot = N - n_i_dot
    n_dot_0 = N - n_dot_j
    n0_dot_mat = n0_dot.unsqueeze(1).expand(-1, n_features)
    n_dot_0_mat = n_dot_0.unsqueeze(0).expand(n_features, -1)
    denominator_prod = n_ii_mat * n_jj_mat * n0_dot_mat * n_dot_0_mat
    epsilon = 1e-12
    denominator = torch.sqrt(torch.clamp(denominator_prod, min=epsilon))
    numerator = (n11 * n00) - (n10 * n01)
    phi_matrix = numerator / denominator
    phi_matrix[denominator_prod < epsilon] = 0.0
    phi_matrix = torch.clamp(phi_matrix, -1.0, 1.0)
    torch.diagonal(phi_matrix).fill_(-torch.inf)
    print("  Phi matrix calculation finished.")
    return phi_matrix

# --- Validation (unchanged) ---
if EMBEDDING_COL not in df.columns:
    raise ValueError(f"DataFrame is missing the required column: '{EMBEDDING_COL}'.")
if 'cooc_embeddings_matrix' not in locals():
    raise NameError("Co-occurrence matrix 'cooc_embeddings_matrix' is not defined.")
if 'N_total_chunks' not in locals():
    raise NameError("Variable 'N_total_chunks' is not defined.")
if cooc_embeddings_matrix.shape[0] != len(df) or cooc_embeddings_matrix.shape[1] != len(df):
    raise ValueError("Shape of cooc_matrix must match the number of features in the DataFrame.")
interpretation_col = 'description' if 'description' in df.columns else 'interpretation'
if interpretation_col not in df.columns:
    print(f"Warning: DataFrame does not have '{interpretation_col}' column. Cannot show interpretations.")

# --- Main Calculation ---
print("Starting Phi coefficient (CUDA) and Cosine Similarity calculation…")
start_time = time.time()

# 1. Prepare Semantic Embeddings
embeddings_np = np.stack(df[EMBEDDING_COL].values).astype(np.float32)
n_features = embeddings_np.shape[0]
original_indices = df.index.to_numpy()

# 2. Semantic similarity matrix
print("Calculating semantic similarities…")
semantic_sim_matrix = cosine_similarity(embeddings_np)
np.fill_diagonal(semantic_sim_matrix, -np.inf)  # Ignore self‑similarity

# 3. Phi coefficient matrix (functional)
phi_sim_matrix_gpu = calculate_phi_matrix_cuda(cooc_embeddings_matrix, N_total_chunks, device)

# 4. Top‑N neighbours for each similarity type
print(f"Finding Top {TOP_N} neighbours for both types…")
top_n_indices_emb = np.argsort(-semantic_sim_matrix, axis=1)[:, :TOP_N]
top_n_indices_phi = torch.argsort(phi_sim_matrix_gpu, dim=1, descending=True)[:, :TOP_N].cpu().numpy()

del phi_sim_matrix_gpu, semantic_sim_matrix  # Free memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# 5. Calculate **NDCG** between the two rankings for each feature
print("Calculating NDCG of neighbour rankings…")
ndcg_scores = {}
log2 = np.log2  # local alias (tiny speed gain)
for i in tqdm(range(n_features), desc="Calculating NDCG"):
    current_original_index = original_indices[i]

    emb_neighbors = top_n_indices_emb[i]
    phi_neighbors = top_n_indices_phi[i]

    # Relevance is highest for rank 0, lower for later — simple linear relevance works well
    # rel = TOP_N, TOP_N‑1, …, 1
    relevance = {idx: 1.1**(TOP_N - rank) - 1 for rank, idx in enumerate(emb_neighbors)}
    
    # --- DCG of the Phi ranking ---
    dcg = 0.0
    for rank, idx in enumerate(phi_neighbors):
        rel = relevance.get(idx, 0)  # 0 relevance if not in the embedding top‑N
        if rel:
            dcg += rel / log2(rank + 2)  # +2 because rank starts at 0

    # --- Ideal DCG (IDCG) — embedding ranking is already ideal ---
    idcg = 0.0
    for rank, idx in enumerate(emb_neighbors):
        rel = relevance[idx]
        idcg += rel / log2(rank + 2)

    ndcg = dcg / idcg if idcg > 0 else 0.0  # Normalised 0–1
    ndcg_scores[current_original_index] = ndcg

# 6. Add results to DataFrame
print("Storing NDCG scores in DataFrame…")
df[OUTPUT_COL] = df.index.map(ndcg_scores)

end_time = time.time()
print(f"\nCalculation finished in {end_time - start_time:.2f} seconds.")

# 7. Display results
print(f"\nDataFrame with NDCG ('{OUTPUT_COL}'):")
display(df[[interpretation_col, OUTPUT_COL]].head())

print(f"\nSummary statistics for '{OUTPUT_COL}':")
print(df[OUTPUT_COL].describe())

print(f"\nTop 5 Features by '{OUTPUT_COL}':")
display(df.nlargest(5, OUTPUT_COL)[[interpretation_col, OUTPUT_COL]])

print(f"\nBottom 5 Features by '{OUTPUT_COL}':")
display(df.nsmallest(5, OUTPUT_COL)[[interpretation_col, OUTPUT_COL]])

# --- Optional: Plotting versus another score column ---
if 'scores' in df.columns:
    import matplotlib.pyplot as plt
    from scipy.stats import pearsonr, spearmanr

    x_column = 'scores'
    y_column = OUTPUT_COL

    x_data = pd.to_numeric(df[x_column], errors='coerce')
    y_data = pd.to_numeric(df[y_column], errors='coerce')
    valid = ~np.isnan(x_data) & ~np.isnan(y_data)
    if valid.sum() > 1:
        x_clean = x_data[valid]
        y_clean = y_data[valid]
        pearson_r, p_pearson = pearsonr(x_clean, y_clean)
        spearman_r, p_spearman = spearmanr(x_clean, y_clean)

        fig, ax = plt.subplots(figsize=(9, 7))
        ax.scatter(x_clean, y_clean, alpha=0.4, s=15, edgecolors='none')
        ax.set_title(f'Relationship between "{x_column}" and "{y_column}»\n'
                     f'Pearson r = {pearson_r:.3f} (p={p_pearson:.2g}) | '
                     f'Spearman ρ = {spearman_r:.3f} (p={p_spearman:.2g})', fontsize=12)
        ax.set_xlabel(x_column)
        ax.set_ylabel(y_column)
        ax.grid(True, linestyle='--', alpha=0.6)
        plt.tight_layout()
        plt.show()
    else:
        print("\nNot enough valid data points to calculate correlation or plot.")

Using CUDA device: NVIDIA GeForce RTX 5090
Starting Phi coefficient (CUDA) and Cosine Similarity calculation…


/home/mmezzanzanica/project/scoring_autoint_align/env/lib/python3.12/site-packages/torch/cuda/__init__.py:287: UserWarning: 
NVIDIA GeForce RTX 5090 with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_70 sm_75 sm_80 sm_86 sm_90.
If you want to use the NVIDIA GeForce RTX 5090 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


Calculating semantic similarities…
  Calculating Phi matrix on device: cuda


RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
df[df['id']==6954]

,id,description,Original_pearson_Corr,Max_Corr_ID,Max_Pearson_Corr,W_dec,embedding,cooc_embedding,scores,ndcg_emb_phi
6954,6954,technical terms and definitions related to mat...,0.023531,16136,0.069549,"[0.012116738, -0.011803359, -0.019546265, -0.0...","[0.04812520369887352, 0.011766992509365082, -0...","[18636, 16642, 10622, 17625, 8815, 10450, 2020...",0.673077,0.001018


In [ ]:
import pandas as pd
from scipy.stats import pearsonr, spearmanr, kendalltau
import numpy as np

# --- Configuration ---
col1_name = 'scores'          # Adjust if your other metric column is named differently
col2_name = 'ndcg_emb_phi'    # <- NEW: the NDCG column

# --- Check if columns exist ---
if col1_name not in df.columns or col2_name not in df.columns:
    print(f"Error: One or both columns ('{col1_name}', '{col2_name}') not found in the DataFrame.")
else:
    # Drop NaNs in either column
    clean = df[[col1_name, col2_name]].dropna()
    if len(clean) < 2:
        print(f"Need at least 2 data points to compute correlations; have {len(clean)}.")
    else:
        print(f"Calculating correlations between '{col1_name}' and '{col2_name}' using {len(clean)} non‑NaN pairs.")
        print("-" * 40)
        # Pearson
        pearson_corr, pearson_p = pearsonr(clean[col1_name], clean[col2_name])
        print(f"Pearson r : {pearson_corr:.4f}\t(p={pearson_p:.4g})")
        print("-" * 40)
        # Spearman
        spearman_corr, spearman_p = spearmanr(clean[col1_name], clean[col2_name])
        print(f"Spearman ρ: {spearman_corr:.4f}\t(p={spearman_p:.4g})")
        print("-" * 40)
        # Kendall's τ
        kendall_corr, kendall_p = kendalltau(clean[col1_name], clean[col2_name])
        print(f"Kendall τ : {kendall_corr:.4f}\t(p={kendall_p:.4g})")
        print("-" * 40)

Calculating correlations between 'scores' and 'ndcg_emb_phi' using 16382 non‑NaN pairs.
----------------------------------------
Pearson r : 0.2231	(p=6.931e-184)
----------------------------------------
Spearman ρ: 0.2853	(p=1.86e-304)
----------------------------------------
Kendall τ : 0.1972	(p=2.856e-299)
----------------------------------------


In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML

# --- Configuration ---
SCORE_COL = 'scores'         # Your existing score column
NDCG_COL  = 'ndcg_emb_phi'   # The new NDCG column
INTERPRETATION_COL = 'description'  # Change if needed
NUM_EXAMPLES_TO_SHOW = 30

# --- Quantile thresholds ---
HIGH_Q = 0.90  # Top 10 %
LOW_Q  = 0.10  # Bottom 10 %

# --- Sanity checks ---
required_cols = ["id", SCORE_COL, NDCG_COL]
if INTERPRETATION_COL not in df.columns:
    if 'interpretation' in df.columns:
        INTERPRETATION_COL = 'interpretation'
        print("Using 'interpretation' column instead of 'description'.")
    else:
        print("Warning: No interpretation column found; tables will omit it.")

for col in required_cols:
    if col not in df.columns:
        raise ValueError(f"DataFrame is missing required column: {col}")

# --- Clean data & derive thresholds
clean_df = df.dropna(subset=[SCORE_COL, NDCG_COL])
if len(clean_df) < 10:
    print(f"Warning: Only {len(clean_df)} non‑NaN rows; quantiles may be unstable.")

hi_score_thr = clean_df[SCORE_COL].quantile(HIGH_Q)
lo_score_thr = clean_df[SCORE_COL].quantile(LOW_Q)
hi_ndcg_thr  = clean_df[NDCG_COL].quantile(HIGH_Q)
lo_ndcg_thr  = clean_df[NDCG_COL].quantile(LOW_Q)

print("-" * 50)
print("Thresholds Used for Defining Quadrants:")
print(f"High '{SCORE_COL}' (≥ {hi_score_thr:.4f}) — top 10 %")
print(f"Low  '{SCORE_COL}' (≤ {lo_score_thr:.4f}) — bottom 10 %")
print(f"High '{NDCG_COL}' (≥ {hi_ndcg_thr:.4f}) — top 10 %")
print(f"Low  '{NDCG_COL}' (≤ {lo_ndcg_thr:.4f}) — bottom 10 %")
print("-" * 50)

# --- Boolean masks
is_hi_score = clean_df[SCORE_COL] >= hi_score_thr
is_lo_score = clean_df[SCORE_COL] <= lo_score_thr
is_hi_ndcg  = clean_df[NDCG_COL] >= hi_ndcg_thr
is_lo_ndcg  = clean_df[NDCG_COL] <= lo_ndcg_thr

indices_hh = clean_df[is_hi_score & is_hi_ndcg].index.tolist()
indices_hl = clean_df[is_hi_score & is_lo_ndcg].index.tolist()
indices_lh = clean_df[is_lo_score & is_hi_ndcg].index.tolist()
indices_ll = clean_df[is_lo_score & is_lo_ndcg].index.tolist()

# --- Helper to display each quadrant

def show_quadrant(name, indices, base_df, score_col, ndcg_col, interp_col, n_show=10):
    print(f"\n— {name} — ({len(indices)} features)")
    if not indices:
        print("No items in this quadrant.")
        return
    # Sort for readability
    if "Low" in name and ndcg_col in name:
        sorted_idx = base_df.loc[indices].sort_values(ndcg_col).index
    elif "High" in name and ndcg_col in name:
        sorted_idx = base_df.loc[indices].sort_values(ndcg_col, ascending=False).index
    else:
        sorted_idx = indices
    cols = [score_col, ndcg_col]
    if interp_col in base_df.columns:
        cols = [interp_col] + cols
    cols.insert(0, "id")
    html = base_df.loc[sorted_idx[:n_show], cols].to_html(index=True, escape=False)
    display(HTML(html))

show_quadrant(f"High {SCORE_COL} & High {NDCG_COL}", indices_hh, df, SCORE_COL, NDCG_COL, INTERPRETATION_COL, NUM_EXAMPLES_TO_SHOW)
show_quadrant(f"High {SCORE_COL} & Low  {NDCG_COL}", indices_hl, df, SCORE_COL, NDCG_COL, INTERPRETATION_COL, NUM_EXAMPLES_TO_SHOW)
show_quadrant(f"Low  {SCORE_COL} & High {NDCG_COL}", indices_lh, df, SCORE_COL, NDCG_COL, INTERPRETATION_COL, NUM_EXAMPLES_TO_SHOW)
show_quadrant(f"Low  {SCORE_COL} & Low  {NDCG_COL}", indices_ll, df, SCORE_COL, NDCG_COL, INTERPRETATION_COL, NUM_EXAMPLES_TO_SHOW)


--------------------------------------------------
Thresholds Used for Defining Quadrants:
High 'scores' (≥ 1.0000) — top 10 %
Low  'scores' (≤ 0.6000) — bottom 10 %
High 'ndcg_emb_phi' (≥ 0.2951) — top 10 %
Low  'ndcg_emb_phi' (≤ 0.0006) — bottom 10 %
--------------------------------------------------

— High scores & High ndcg_emb_phi — (605 features)


,id,description,scores,ndcg_emb_phi
4688,4688,terms and phrases related to monetary transactions and payments,1.0,0.943712
11014,11014,instances where the actions of replacing or substituting something are discussed,1.0,0.907036
770,770,varying expressions of fear and anxiety,1.0,0.896276
1840,1840,phrases related to substitutions and replacements,1.0,0.887752
7028,7028,phrases indicating conditionality or exceptions in statements,1.0,0.883812
11196,11196,terms related to management in various contexts,1.0,0.877673
3695,3695,references to language proficiency and multilingualism,1.0,0.867384
15986,15986,phrases that express exceptions or conditional statements,1.0,0.865819
5197,5197,terms related to heating and warmth in various contexts,1.0,0.865643
4202,4202,"occurrences of the word ""Also"" with varying frequencies",1.0,0.864726



— High scores & Low  ndcg_emb_phi — (156 features)


,id,description,scores,ndcg_emb_phi
13390,13390,references to specific spiritual or religious figures and practices,1.0,0.000000e+00
10306,10306,mathematical formatting structures within the text,1.0,6.008934e-107
2208,2208,terms indicating present status or ongoing activity,1.0,1.921081e-99
9936,9936,"occurrences of the term ""typeof"" in various contexts",1.0,5.349172e-88
1000,1000,function calls and references in programming languages,1.0,6.157942e-88
12964,12964,"instances of the word ""from"" in various contexts",1.0,2.836481e-51
11150,11150,instances of time and personal references,1.0,3.204906e-36
3565,3565,references to scholarly work and academic contributions,1.0,7.308226e-32
979,979,notations related to mathematical labels and equations,1.0,9.299057e-28
1161,1161,expressions related to equality and fairness in various contexts,1.0,7.245699e-27



— Low  scores & High ndcg_emb_phi — (29 features)


,id,description,scores,ndcg_emb_phi
7649,7649,phrases related to online shopping and promotional offers,0.555556,0.299023
1606,1606,character sequences and codes often associated with technical or programming contexts,0.583333,0.314813
4581,4581,references and citations in a text,0.375000,0.317613
7480,7480,phrases that indicate types or categories of items or concepts,0.557692,0.333862
788,788,concepts related to role-playing game mechanics and character customization,0.550000,0.338541
12932,12932,"phrases and terms related to value, benefits, and impacts of actions or situations",0.444444,0.354367
14917,14917,references to authoritarian regimes and their oppressive actions,0.392857,0.365705
12907,12907,references to puzzle games and their mechanics,0.388889,0.369081
12558,12558,references to sleep disorders and their related symptoms,0.500000,0.372572
11916,11916,references to livestock and related veterinary conditions,0.388889,0.373069



— Low  scores & Low  ndcg_emb_phi — (416 features)


,id,description,scores,ndcg_emb_phi
6173,6173,structural elements of code or programming syntax,0.225000,0.000000e+00
10625,10625,negative sentiment or criticism,0.500000,1.110755e-286
6887,6887,"terms related to leadership styles in a professional context, particularly in nursing management",0.555556,2.892549e-222
13486,13486,terms associated with regulatory bodies and financial processes,0.465909,1.550429e-185
11077,11077,"terms related to cellular and molecular biology, particularly in the context of research studies involving treatment and effects on tumors",0.500000,2.804566e-147
10055,10055,significant numerical data or statistics,0.500000,3.211242e-132
6879,6879,spiritual or ritualistic practices involving offerings and purification,0.583333,3.151864e-123
12830,12830,symbols and formatting related to formal mathematical expressions or legal documents,0.361111,1.023096e-121
579,579,references to specific individuals or legal contexts,0.529412,2.953165e-104
10160,10160,HTML elements and attributes related to user interface interactions and visual effects,0.000000,3.801172e-88


In [ ]:
#4284
#11077 vinciamo di brutto


In [ ]:
clean_df[['id', 'description', 'scores', 'ndcg_emb_phi']].to_csv('clean_df.csv', index=False)

In [ ]:
# Assegna ciascuna riga a un decile (qcut calcola automaticamente i bin sui quantili)
clean_df['phi_decile'] = pd.qcut(clean_df['ndcg_emb_phi'], q=10, labels=False, duplicates='drop')

# Campionamento: uno per decile
sampled_df = clean_df.groupby('phi_decile').apply(lambda x: x.sample(1, random_state=42)).reset_index(drop=True)

# Colonne desiderate
final_sample = sampled_df[['id', 'description', 'ndcg_emb_phi', 'phi_decile']].sort_values('phi_decile')
final_sample = final_sample[['id', 'description', 'ndcg_emb_phi']]
# Output
final_sample

/var/folders/b2/vgr8pw4s1hz28zp9077tfpyw0000gn/T/ipykernel_9322/688803868.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['phi_decile'] = pd.qcut(clean_df['ndcg_emb_phi'], q=10, labels=False, duplicates='drop')
/var/folders/b2/vgr8pw4s1hz28zp9077tfpyw0000gn/T/ipykernel_9322/688803868.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = clean_df.groupby('phi_decile').apply(lambda x: x.sample(1, random_state=42)).reset_index(drop=True

,id,description,ndcg_emb_phi
0,6954,technical terms and definitions related to mat...,1.559755e-15
1,6807,"actions related to development, improvement, a...",5.083827e-03
2,6616,ellipses or continuative punctuation and elem...,3.392980e-02
3,6868,mathematical inequalities and set notations i...,6.645351e-02
4,6480,references to decision-making and choices,7.881524e-02
5,6906,elements related to data structure or format ...,1.180365e-01
6,6630,terms and concepts related to medical imaging...,1.505439e-01
7,6739,entities related to pharmaceutical companies a...,1.647480e-01
8,6882,financial metrics and information related to c...,2.567771e-01
9,6766,references to ownership and the role of owners,3.508244e-01


In [ ]:
# Mischia le righe del campione
final_sample_shuffled = final_sample.sample(frac=1, random_state=42).reset_index(drop=True)

# Visualizza
final_sample_shuffled = final_sample_shuffled[['id', 'description']]

In [ ]:
# Inserisci qui il vero neuronpedia_id se lo conosci
neuronpedia_id = 'gemma-2-9b/8-gemmascope-res-16k'
layer = 8

def build_url(latent_idx, neuronpedia_id, layer):
    return f'https://neuronpedia.org/{neuronpedia_id}/{latent_idx}?embed=true&embedexplanation=true&embedplots=true&embedtest=true&height=300'

# Applichiamo la funzione al dataframe
final_sample_shuffled['url'] = final_sample_shuffled['id'].apply(lambda idx: build_url(idx, neuronpedia_id, layer))

In [ ]:
final_sample_shuffled

,id,description,url
0,6882,financial metrics and information related to c...,https://neuronpedia.org/gemma-2-9b/8-gemmascop...
1,6807,"actions related to development, improvement, a...",https://neuronpedia.org/gemma-2-9b/8-gemmascop...
2,6906,elements related to data structure or format ...,https://neuronpedia.org/gemma-2-9b/8-gemmascop...
3,6954,technical terms and definitions related to mat...,https://neuronpedia.org/gemma-2-9b/8-gemmascop...
4,6739,entities related to pharmaceutical companies a...,https://neuronpedia.org/gemma-2-9b/8-gemmascop...
5,6616,ellipses or continuative punctuation and elem...,https://neuronpedia.org/gemma-2-9b/8-gemmascop...
6,6766,references to ownership and the role of owners,https://neuronpedia.org/gemma-2-9b/8-gemmascop...
7,6480,references to decision-making and choices,https://neuronpedia.org/gemma-2-9b/8-gemmascop...
8,6868,mathematical inequalities and set notations i...,https://neuronpedia.org/gemma-2-9b/8-gemmascop...
9,6630,terms and concepts related to medical imaging...,https://neuronpedia.org/gemma-2-9b/8-gemmascop...


In [ ]:
final_sample['url'][0]

'https://neuronpedia.org/gemma-2-9b/8-gemmascope-res-16k/6954?embed=true&embedexplanation=true&embedplots=true&embedtest=true&height=300'

In [ ]:
final_sample_shuffled.to_excel("/Users/antonioserino/Documents/sampled_neurons.xlsx", index=False)